## ハンズオン課題3 解答

**実行するには，メニューの「PLAYGROUNDで開く」を押してください。（Google アカウントが必要です。）**

In [0]:
# janome をインストール
!pip install janome

# https://ipafont.ipa.go.jp/old/ipafont/download.html から TTFファイル「IPA P ゴシック」をダウンロードして手元で解凍し，`ipagp.ttf` ファイルを Google Colab にアップロード
from google.colab import files
uploaded = files.upload()

### 3-1 解答例

In [0]:
# janome-tutorial/hands-on/data/kazeno_matasaburo_utf8.txt を Google Colab にアップロード
from google.colab import files
uploaded = files.upload()

In [0]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
import sys

MAX_WORDS = 20

def wc(file):
    with open(file, encoding='utf8') as f:
        """
        ここに処理を書く。
        戻り値として，(単語, 出現回数) のタプルのリストを返す。
        """
        token_filters = [TokenCountFilter(sorted=True)]
        a = Analyzer(tokenizer=Tokenizer(), token_filters=token_filters)
        text = f.read()
        return list(a.analyze(text))[:MAX_WORDS]
      
counts = wc('kazeno_matasaburo_utf8.txt')
for k, v in counts:
    print('%s\t%d' % (k, v))

### 3-2 解答例

In [0]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *

MAX_WORDS = 20

def wc(file, pos=[]):
    with open(file, encoding='utf8') as f:
        if not pos:
            token_filters = [TokenCountFilter(sorted=True)]
        else:
            token_filters = [POSKeepFilter(pos), TokenCountFilter(sorted=True,att='base_form')]
        a = Analyzer(tokenizer=Tokenizer(), token_filters=token_filters)
        text = f.read()
        return list(a.analyze(text))[:MAX_WORDS]

# 名詞と動詞に絞ってカウント
counts = wc('kazeno_matasaburo_utf8.txt', ['名詞','動詞'])
for k, v in counts:
    print('%s\t%d' % (k, v))

### 3-3 解答例

In [0]:
# StopWordFilter の実装
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *

class StopWordFilter(TokenFilter):
  def __init__(self, word_list=[], word_list_file=''):
    self.word_list = []
    if word_list:
      self.word_list = word_list
    elif word_list_file:
      with open(word_list_file) as f:
        for line in f:
          word = line.strip()
          self.word_list.append(word)
    
  def apply(self, tokens):
      for token in tokens:
        if any(token.base_form == word for word in self.word_list):
          continue
        yield token

In [0]:
# stop word をリストで指定する
token_filters = [StopWordFilter(word_list=['プログラミング'])]
a = Analyzer(tokenizer=Tokenizer(), token_filters=token_filters)
for token in a.analyze('Pythonは人気の高いプログラミング言語です。'):
  print(token)

In [0]:
# stop word をファイルで指定する
!echo "プログラミング" > stop_words.txt
!echo "Python" >> stop_words.txt

token_filters = [StopWordFilter(word_list_file='stop_words.txt')]
a = Analyzer(tokenizer=Tokenizer(), token_filters=token_filters)
for token in a.analyze('Pythonは人気の高いプログラミング言語です。'):
  print(token)

### 3-4 解答例

In [0]:
!echo 又三郎,名詞-カスタム,マタサブロー > 'udic.csv'

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def create_analyzer(udic_file, udic_type):
    tokenizer = Tokenizer(udic_file, udic_type=udic_type, udic_enc='utf8') if udic_file else Tokenizer()
    char_filters = [RegexReplaceCharFilter('《.*?》', '')]
    token_filters = [POSKeepFilter(['名詞','形容詞','形容動詞','感動詞']), POSStopFilter(['名詞,非自立','名詞,代名詞']), ExtractAttributeFilter('base_form')]
    return Analyzer(char_filters, tokenizer, token_filters)

def split_text(src, out, udic_file='udic.csv', udic_type='simpledic'):
    """
    src で渡されたファイルを読み，単語分割して out に書き出します。
    """
    a = create_analyzer(udic_file, udic_type)
    with open(src, encoding='utf8') as f1:
        with open(out, mode='w', encoding='utf8') as f2:
            for line in f1:
                tokens = list(a.analyze(line))
                f2.write('%s\n' % ' '.join(tokens))

def show_wordcloud(file):
    with open(file, encoding='utf8') as f:
        text = f.read()
        wordcloud = WordCloud(font_path='ipagp.ttf', background_color='white', width=1024, height=674).generate(text)
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.figure()
        plt.show()
        
split_text('kazeno_matasaburo_utf8.txt', 'words.txt')
show_wordcloud('words.txt')